## Os dejo aquí cómo se usa mi librería (mergerML):
#### Lo único que tenéis que hacer para que funcione es tener el archivo `mergerml.py` en la misma carpeta que el Dash.

In [1]:
pdf_que_queremos_procesar = 'DEMO_PDF/BANKIA_2020.pdf' # Este path lo sacais del PDF que metan en el Dash

### Path a los modelos que necesitamos:

In [2]:
path_word2vec_model   = 'MODELS/modelo_word2vec_100.model'
path_clustering_model = 'MODELS/clustering_pipeline.joblib'
path_merger_model     = 'MODELS/merger_predict.joblib'

### Lectura del PDF:

In [3]:
from mergerml import process_PDFs

In [4]:
rawdata, tiempo_que_tarda = process_PDFs([pdf_que_queremos_procesar], # Van en una lista, aunque solo haya un PDF
                                          esp=False, # Si está en ESPAÑOL (esta opción tenemos que ponerla en el DASH)
                                          verbose=2, # Si queremos que nos haga prints por terminal mientras procesa
                                          save_path=None # Si queremos guardar los resultados a algun lado
                                         )

Cronometrando lectura de PDFs...

BANKIA_2020.pdf
--- Tiempo de lectura (s): 2.1870062351226807
Preparando dataframe...
Tokenizando frases...


100%|███████████████████████████████████████| 186/186 [00:00<00:00, 4596.90it/s]


In [5]:
rawdata.sample(10)

sentences
bank   year page sentence                                                   
BANKIA 2020 41   41                                    range of user queries
            157  31                       Tax Practices CBTP which includes 
            16   4          since Strategy implemented for setting an int...
            53   4          For Bankia using technology to manage the cus...
            3    3               Banking as a solution to the health crisis 
            45   30                         DevSecOps processes has made it 
            42   57                      organisational structure and other 
            174  2          During the bank accepted recovery agreements ...
            134  31                     annual accounts the distribution of 
            154  15                     and developing internal regulations

In [6]:
tiempo_que_tarda

{'lectura': 2.3141095638275146}

### Análisis de Sentimiento:

In [5]:
from mergerml import get_phrase_sentiment

In [6]:
data, tiempo_que_tarda_sentiment = get_phrase_sentiment(rawdata.sample(200), # El resultado de la función anterior
                                                        min_words=5, # Esto podríamos ponerlo como opción en el Dash
                                                        save_sentiment_path=None, # Si queremos guardar los resultados a algun lado
                                                       )

Cronometrando procesado de sentiment de frases...


100%|█████████████████████████████████████| 121/121 [00:00<00:00, 253122.59it/s]


In [9]:
data.sample(10)

sentences  \
bank   year page sentence                                                      
BANKIA 2020 117  45                      companies segment to allow them to    
            168  8         RISK MANAGEMENT CREDIT RISK PROFILE MORTGAGE B...   
            143  2          Directors remuneration comprises a fixed and ...   
            122  27                  million euros at yearend thanks to the    
            169  138                    with the financing as well as clear    
            117  93                          turnover as a result groups of    
            48   28                     facilitated more than billion euros    
                 54                    support measures such as the payment    
            162  72                                 standard PCI DSS is high   
            153  4          These risks are subject to general and specif...   

                           length  negative   neutral  positive  
bank   year page sentence                                        
BANKIA 2020 117  45             6  0.000061  0.999883  0.000056  
            168  8             23  0.000069  0.999883  0.000048  
            143  2              9  0.000079  0.999882  0.000039  
            122  27             7  0.000105  0.999669  0.000226  
            169  138            7  0.000059  0.999889  0.000051  
            117  93             6  0.000053  0.999834  0.000114  
            48   28             5  0.000057  0.999875  0.000069  
                 54             6  0.000061  0.999892  0.000047  
            162  72             5  0.730953  0.264747  0.004300  
            153  4             12  0.000073  0.999886  0.000042

In [10]:
tiempo_que_tarda_sentiment

{'sentiment': 99.84928917884827}

### Predicción:

In [7]:
from mergerml import predict_merger

In [8]:
probabilidad_adquisicion, tiempo_que_tarda_predict, sentimiento_de_secciones = predict_merger(data, 
                                                                                              path_word2vec_model,  
                                                                                              path_clustering_model,
                                                                                              path_merger_model,    
                                                                                              timeit=True
                                                                                             )

Tokenizing...


100%|█████████████████████████████████████| 121/121 [00:00<00:00, 122973.29it/s]


Aplicando Word2Vec...


100%|███████████████████████████████████████| 121/121 [00:00<00:00, 6532.43it/s]

Aplicando Clustering...


Preparando Dataset...
Cargando Modelo...
Haciendo Predicción...


In [13]:
probabilidad_adquisicion

array([0.28264401])

In [14]:
tiempo_que_tarda_predict

{'prediction': 0.7640392780303955}

In [9]:
sentimiento_de_secciones

neg       pos
bank   year                              
BANKIA 2020 cluster_0  0.000082  0.000071
            cluster_1  0.000113  0.199858
            cluster_2  0.000083  0.069157
            cluster_3  0.037619  0.094907
            cluster_4  0.040081  0.151629

In [12]:
sentimiento_de_secciones.reset_index().rename(columns={'level_2':'clusters'})[['clusters','neg','pos']].to_dict('records')

[{'clusters': 'cluster_0',
  'neg': 8.215275011025369e-05,
  'pos': 7.050571730360389e-05},
 {'clusters': 'cluster_1',
  'neg': 0.00011338798503857106,
  'pos': 0.19985784590244293},
 {'clusters': 'cluster_2',
  'neg': 8.280184556497261e-05,
  'pos': 0.06915676593780518},
 {'clusters': 'cluster_3',
  'neg': 0.03761924430727959,
  'pos': 0.09490697830915451},
 {'clusters': 'cluster_4',
  'neg': 0.040081173181533813,
  'pos': 0.15162859857082367}]

In [25]:
print(sentimiento_de_secciones.to_latex())

\begin{tabular}{lllrr}
\toprule
       &      &           &       neg &       pos \\
bank & year & {} &           &           \\
\midrule
BANKIA & 2020 & cluster\_0 &  0.000069 &  0.000053 \\
       &      & cluster\_1 &  0.029282 &  0.089355 \\
       &      & cluster\_2 &  0.004674 &  0.020489 \\
       &      & cluster\_3 &  0.026072 &  0.057648 \\
       &      & cluster\_4 &  0.020933 &  0.114990 \\
\bottomrule
\end{tabular}

